- https://github.com/ksopyla/awesome-nlp-polish
- Sentiment: https://pypi.org/project/sentimentpl/
- Auto correct: https://github.com/filyp/autocorrect
- other: https://github.com/sdadas/polish-nlp-resources

In [1]:
import pandas as pd
import numpy as np
import spacy
from sentimentpl.models import SentimentPLModel
from autocorrect import Speller
from transformers import HerbertTokenizer, RobertaModel
#import polyglot

ModuleNotFoundError: No module named 'sentimentpl'

In [2]:
nlp_core = spacy.load("pl_core_news_lg")

OSError: [E050] Can't find model 'pl_core_news_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [3]:
model = SentimentPLModel(from_pretrained='latest')

In [6]:
# spell = Speller('pl')

In [7]:
tokenizer_herbert = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model_roberta = RobertaModel.from_pretrained("allegro/herbert-klej-cased-v1")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMTokenizer'. 
The class this function is called from is 'HerbertTokenizer'.


In [ ]:
#nlp_pl = spacy.load('pl_spacy_model') # or spacy.load('pl_spacy_model_morfeusz')

In [ ]:
doc = nlp_core("Nienawidzę tego robić, tym bardziej że robię to codziennie. Ale przynajmniej mam łatwą pracę w Google.")
lemmas_list = []
tokens_list = []
sentiments_list = []
embeddings_list = []

error_n = 0

adj_n = 0
adv_n = 0
noun_n = 0
ent_n = 0

print(doc.text)
print(f'Sentiment: {model(doc.text).item()}\n')

for i, sent in enumerate(doc.sents):
    s = model(sent.text).item()
    print(f'Sentiment sentence {i}: {s}')  
    sentiments_list.append(s)
print()
print(f'AVG sentiment of sentences: {np.mean(sentiments_list)}\n')


for token in doc:
    #print(token.text, token.pos_, token.dep_, token.lemma_)
    if token.pos_ not in ['SPACE', 'PUNCT']:
        lemmas_list.append(token.lemma_)
        tokens_list.append(token.text)
        corrected = spell(token.text)
        if corrected != token.text:
            error_n += 1
    
    if token.pos_ == 'ADJ': 
        adj_n += 1
    elif token.pos_ == 'ADV':
        adv_n += 1
    elif token.pos_ == 'NOUN':
        noun_n += 1
        
for ent in doc.ents:
    #print(ent.text, ent.start_char, ent.end_char, ent.label_)
    ent_n += 1
    
tokens_list = list(set(tokens_list))
lemmas_list = list(set(lemmas_list))

print('Unique words:', len(tokens_list))
print('Unique lemmas:', len(lemmas_list), '\n')
print('Errors:', error_n)
print('Named entities:', ent_n, '\n')
print('ADJ:', adj_n, adj_n/len(tokens_list))
print('ADV:', adv_n, adv_n/len(tokens_list))
print('NOUN:', noun_n, noun_n/len(tokens_list))


In [95]:
encoded_input = tokenizer_herbert.encode(doc.text, return_tensors="pt")
outputs = model_roberta(encoded_input)
embeddings = outputs['pooler_output'].detach().numpy()